# ALL IMPORTS
import pandas as pd
from functools import reduce
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import numpy as np
from datetime import timedelta
from statsmodels.graphics.tsaplots import plot_acf
import seaborn as sns
import scipy.stats as stats

# DATA LOADING AND TRANSFORMING
df = pd.read_csv("D:/data/MORTGAGE30US.csv")

# Step 2: Convert 'observation_date' to datetime
df['date'] = pd.to_datetime(df['observation_date'])

# Step 3: Set the date to the first of each month
df['date'] = df['date'].dt.to_period('M').dt.to_timestamp()

# Step 4: Rename the column for clarity
df = df.rename(columns={'MORTGAGE30US': 'mortgage_rate'})

# Step 5: Convert mortgage_rate to numeric (some entries may be '.')
df['mortgage_rate'] = pd.to_numeric(df['mortgage_rate'], errors='coerce')

# Step 6: Group by the cleaned DATE column and compute monthly average
monthly_avg_df = df.groupby('date')['mortgage_rate'].mean().reset_index()

# Step 7: Save the cleaned data
monthly_avg_df.to_csv("cleaned_mortgage_data.csv", index=False)

# Optional: Show number of monthly records
print("✅ Total monthly records:", len(monthly_avg_df))

print(monthly_avg_df.head())

# Load and clean each dataset
cs = pd.read_csv('D:/data/CSUSHPISA.csv', parse_dates=['observation_date']).rename(columns={'observation_date': 'date', 'CSUSHPISA': 'home_price_index'})
fed = pd.read_csv('D:/data/FEDFUNDS.csv', parse_dates=['observation_date']).rename(columns={'observation_date': 'date', 'FEDFUNDS': 'fed_funds_rate'})
houst = pd.read_csv('D:/data/HOUST.csv', parse_dates=['observation_date']).rename(columns={'observation_date': 'date', 'HOUST': 'housing_starts'})
unrate = pd.read_csv('D:/data/UNRATE.csv', parse_dates=['observation_date']).rename(columns={'observation_date': 'date', 'UNRATE': 'unemployment_rate'})
cpi = pd.read_csv('D:/data/CPIAUCSL.csv', parse_dates=['observation_date']).rename(columns={'observation_date': 'date', 'CPIAUCSL': 'inflation_cpi'})

# Align all dates to start of the month
cs = align_month_start(cs)
print(cs.head())
fed = align_month_start(fed)
houst = align_month_start(houst)
unrate = align_month_start(unrate)
cpi = align_month_start(cpi)

print("✅ All CSVs loaded and dates aligned to month start.")
print(len(cs),len(fed),len(houst),len(unrate),len(cpi))


# MERGE ALL DATA IN A TABLE AND NORMALIZING
dfs = [cs, fed, houst, unrate, cpi,monthly_avg_df]
df_merged = reduce(lambda left, right: pd.merge(left, right, on='date', how='inner'), dfs)

print(f"✅ Merged DataFrame shape: {df_merged.shape}")
print(df_merged.head())

print(df_merged.isnull().sum())

# Check total missing cells
print("\nTotal missing cells:", df_merged.isnull().sum().sum())
df_merged['cpi_pct_change'] = df_merged['inflation_cpi'].pct_change() * 100
df_merged['housing_starts_pct_change'] = df_merged['housing_starts'].pct_change() * 100
print("normlized cpi and housing_starts")
df_merged.drop(columns=['inflation_cpi', 'housing_starts'], inplace=True)
df_merged.dropna(inplace=True)

# FINDING ALL THE CORRELEATION FACTOR
correlation = df_merged.corr()
print(correlation['home_price_index'].sort_values(ascending=False))
print(len(df_merged))

# PLOTING FIRST PLOT (HOME INDEX PLOT)
df_merged['cpi_pct_change'] = df_merged['inflation_cpi'].pct_change() * 100
df_merged['housing_starts_pct_change'] = df_merged['housing_starts'].pct_change() * 100
print("normlized cpi and housing_starts")
df_merged.drop(columns=['inflation_cpi', 'housing_starts'], inplace=True)
df_merged.dropna(inplace=True)


correlation = df_merged.corr()
print(correlation['home_price_index'].sort_values(ascending=False))
print(len(df_merged))

# ADF TEST
def adf_test(series, title=''):
    print(f'ADF Test: {title}')
    result = adfuller(series.dropna())
    labels = ['ADF Statistic', 'p-value', '# Lags Used', '# Observations Used']
    out = dict(zip(labels, result[0:4]))
    for key, val in out.items():
        print(f"{key}: {val}")
    for key, val in result[4].items():
        print(f'Critical Value {key}: {val}')
    
    if result[1] <= 0.05:
        print("✅ Series is stationary (reject H0)")
    else:
        print("❌ Series is non-stationary (fail to reject H0)")

# Run on home_price_index
adf_test(df_merged['home_price_index'], 'Home Price Index')

# TO MAKE SERIES Stationery SO WE CAN FIT IN TIME SERIES
df_merged['home_price_diff'] = df_merged['home_price_index'].diff()

# Visualize differenced series
plt.figure(figsize=(12, 6))
plt.plot(df_merged['home_price_diff'], label='Differenced Home Price Index')
plt.title("1st Difference of Home Price Index")
plt.xlabel("Date")
plt.ylabel("Differenced Index")
plt.legend()
plt.grid(True)
plt.show()

# Re-run ADF on differenced series
adf_test(df_merged['home_price_diff'], 'Differenced Home Price Index')

# SECOND DIFF 
df_merged['home_price_diff2'] = df_merged['home_price_diff'].diff()

plt.figure(figsize=(12, 6))
plt.plot(df_merged['home_price_diff2'], label='2nd Differenced Home Price Index')
plt.title("2nd Difference of Home Price Index")
plt.xlabel("Date")
plt.ylabel("Differenced Index")
plt.legend()
plt.grid(True)
plt.show()

# Re-run ADF on 2nd difference
adf_test(df_merged['home_price_diff2'], '2nd Differenced Home Price Index')

# FITTING MODEL IN TIME SERIES 
y = df_merged['home_price_index']
exog = df_merged[['fed_funds_rate', 'cpi_pct_change', 'unemployment_rate', 'housing_starts_pct_change','mortgage_rate']]

# Drop NA for alignment
data = pd.concat([y, exog], axis=1).dropna()
y = data['home_price_index']
X = data.drop(columns='home_price_index')

# Fit SARIMAX (ARIMA with exogenous)
model = SARIMAX(y, exog=X, order=(1, 2, 1))  # try (1,2,1) or use auto_arima to guide
results = model.fit(disp=False)
print(results.summary())
pred = results.get_prediction(start=0, end=len(y)-1, exog=X)
pred_mean = pred.predicted_mean

# Plot
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(y, label='Actual', linewidth=2)
plt.plot(pred_mean, label='Fitted', linestyle='--')
plt.title('SARIMAX (1,2,1) Model: Actual vs Fitted')
plt.legend()
plt.grid(True)
plt.show()

# GETTING RMSE VALUE
rmse = mean_squared_error(y, pred_mean, squared=False)
print(f"Train RMSE: {rmse:.2f}")

# EXTRA WORK - GETTING ALL PLOTS TO FIND RESIDULAS AND OUTLIERS AND 1 YEAR FUTURE PREDICTION
# FIRST PLOTING THE GRAPHS FOR ALREADY FIT MODEL
y = df_merged['home_price_index']
exog = df_merged[['fed_funds_rate', 'cpi_pct_change', 'unemployment_rate',
           'housing_starts_pct_change', 'mortgage_rate']]

# Step 4: Fit the SARIMAX model again
model = SARIMAX(y, exog=exog, order=(1, 2, 1))
results = model.fit(disp=False)

# Step 5: Plot Residuals
residuals = results.resid

plt.figure(figsize=(14, 8))
plt.subplot(2, 2, 1)
plt.plot(residuals)
plt.title('Residuals Over Time')

plt.subplot(2, 2, 2)
sns.histplot(residuals, kde=True)
plt.title('Histogram of Residuals')

plt.subplot(2, 2, 3)
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Q-Q Plot')

plt.subplot(2, 2, 4)
plot_acf(residuals.dropna(), ax=plt.gca())
plt.title('ACF of Residuals')

plt.tight_layout()
plt.show()

# Step 6: Forecast 12 Months Ahead
n_forecast = 12
future_dates = pd.date_range(start=df.index[-1] + pd.DateOffset(months=1), periods=n_forecast, freq='M')

# You must provide future values for exogenous variables — here we just use the last row repeated
future_exog = pd.DataFrame([exog.iloc[-1].values] * n_forecast, columns=exog.columns, index=future_dates)

# Forecast
forecast_res = results.get_forecast(steps=n_forecast, exog=future_exog)
forecast_mean = forecast_res.predicted_mean
forecast_ci = forecast_res.conf_int()

# Step 7: Plot Actual vs Forecast
plt.figure(figsize=(12, 6))
plt.plot(y, label="Historical")
plt.plot(forecast_mean, label="Forecast", color='orange')
plt.fill_between(forecast_ci.index, forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1],
                 color='orange', alpha=0.3, label="95% CI")
plt.title("12-Month Home Price Forecast")
plt.xlabel("Date")
plt.ylabel("Home Price Index")
plt.legend()
plt.show()

# RMSE  on a hold-out set
pred = results.fittedvalues
rmse = np.sqrt(mean_squared_error(y[2:], pred[2:]))  
print(f"Train RMSE: {rmse:.2f}")

# CLEANING AN REFITTIG
# Ensure datetime index
df_merged = df_merged.sort_index()
df_merged.index = pd.to_datetime(df_merged.index)

# Step 1: Fit initial model to get residuals
initial_model = SARIMAX(df_merged['home_price_index'], 
                        exog=df_merged[['fed_funds_rate', 'cpi_pct_change', 'unemployment_rate', 
                                        'housing_starts_pct_change', 'mortgage_rate']],
                        order=(1, 2, 1))
initial_results = initial_model.fit()

# Step 2: Calculate residuals and z-scores
residuals = df_merged['home_price_index'] - initial_results.fittedvalues
z_scores = (residuals - residuals.mean()) / residuals.std()

# Step 3: Remove outliers (z > 2.5)
outliers = np.abs(z_scores) > 2.5
df_cleaned = df_merged[~outliers]

# Step 4: Refit model on cleaned data
model_cleaned = SARIMAX(df_cleaned['home_price_index'], 
                        exog=df_cleaned[['fed_funds_rate', 'cpi_pct_change', 'unemployment_rate', 
                                         'housing_starts_pct_change', 'mortgage_rate']],
                        order=(1, 2, 1))
results_cleaned = model_cleaned.fit()

# Step 5: Forecast 12 months ahead
future_steps = 12
last_date = df_cleaned.index[-1]
forecast_index = pd.date_range(start=last_date + timedelta(days=30), periods=future_steps, freq='MS')

# Extend exogenous variables for forecast using the latest known values
last_exog = df_cleaned[['fed_funds_rate', 'cpi_pct_change', 'unemployment_rate', 
                        'housing_starts_pct_change', 'mortgage_rate']].iloc[-1]
future_exog = pd.DataFrame([last_exog]*future_steps, index=forecast_index)

forecast = results_cleaned.get_forecast(steps=future_steps, exog=future_exog)
forecast_mean = forecast.predicted_mean
forecast_ci = forecast.conf_int()
# Step 6: Plot the results
plt.figure(figsize=(12, 6))
plt.plot(df_merged['home_price_index'], label='Historical')
plt.plot(forecast_index, forecast_mean, label='Forecast', color='orange')
plt.fill_between(forecast_index, forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1], 
                 color='orange', alpha=0.3, label='95% CI')
plt.title('12-Month Home Price Forecast (Outliers Removed)')
plt.xlabel('Date')
plt.ylabel('Home Price Index')
plt.legend()
plt.grid(True)
plt.show()

# PLOTING ALL THE GRAPHS 
from statsmodels.graphics.tsaplots import plot_acf
import seaborn as sns
import scipy.stats as stats

residuals_cleaned = df_cleaned['home_price_index'] - results_cleaned.fittedvalues
fig, ax = plt.subplots(2, 2, figsize=(12, 8))

# Residuals over time
ax[0, 0].plot(residuals_cleaned)
ax[0, 0].set_title('Residuals Over Time')

# Histogram
sns.histplot(residuals_cleaned, bins=30, kde=True, ax=ax[0, 1])
ax[0, 1].set_title('Histogram of Residuals')

# QQ plot
stats.probplot(residuals_cleaned.dropna(), dist="norm", plot=ax[1, 0])
ax[1, 0].set_title('Q-Q Plot')

# ACF plot
plot_acf(residuals_cleaned.dropna(), ax=ax[1, 1])
ax[1, 1].set_title('ACF of Residuals')

plt.tight_layout()
plt.show()